In [1]:
import numpy as np
import pandas as pd

In [23]:
def presion_vapor_sat(T):
    if (T>=0):
        es = 611 * np.exp(17.27*T/(T + 237.3))
    else:
        es = 611 * np.exp(21.87*T/(T + 265.5))
    return es 

def latente(T):
    return 2.501 - 0.00236 * T

In [31]:
p_a = 100 # Presión (kPa)
cp = 1.005e-3 # Calor específico a presión constante (MJ/kgK) 
Ra = 287 # Constante de gas (J/kgK)
k = 0.41 # Constante de von Karman
rho_w = 1000 # densidad del agua (kg/m3)

zm = 2
zd = 0
z0 = 10e-5

data = {'u': [2.4, 1.6, 3.2, 0.5, 5],
        'T': [5.2, 12.2, 19.3, 0.7, 9.9],
        'RH': [100, 86, 68, 40, 100],
        'Ts': [7, 8, 15.6, 5.5, 9.9]}

data = pd.DataFrame(data)
data

,u,T,RH,Ts
0,2.4,5.2,100,7.0
1,1.6,12.2,86,8.0
2,3.2,19.3,68,15.6
3,0.5,0.7,40,5.5
4,5.0,9.9,100,9.9


### Cálculo de la tasa de evaporación

In [41]:
E = L = H = []

for i, row in data.iterrows():
    u = row['u']
    T = row['T']
    RH = row['RH']
    Ts = row['Ts']
    
    rho_a = p_a/(Ra * (T + 273.15)) * 1000 # densidad del aire en kg/m3
    kE = 0.622 * rho_a / (p_a * rho_w) * (k/np.log((zm - zd)/z0)) ** 2
    klE = kE * latente(T) * rho_w
    kH = rho_a * cp * (k/np.log((zm - zd)/z0)) ** 2
        
    es = presion_vapor_sat(T)
    e_zm = RH * presion_vapor_sat(Ts)/100
        
    E = E + [kE * u * (es - e_zm) * 3600 * 1000] # tasa de evaporación (mm/hr)
    L = L + [klE * u * (es - e_zm) * 3600] # flujo de calor latente (MJ/hr/m2)
    H = H + [kH * u * (Ts - T) * 3600] # flujo de calor sensible (MJ/hr/m2)

output = pd.DataFrame(dict(E=E, L=L, H=H))
output = pd.concat([data, output], axis=1)
output

,u,T,RH,Ts,E,L,H
0,2.4,5.2,100,7.0,-13.528212,-33.668039,0.033533
1,1.6,12.2,86,8.0,37.396215,92.451222,-0.050883
2,3.2,19.3,68,15.6,151.293894,371.494895,-0.087474
3,0.5,0.7,40,5.5,6.871085,17.173233,0.018935
4,5.0,9.9,100,9.9,0.000000,0.000000,0.000000


Los resultados indican que ocurre condensación en el día 1 y que no hay intercambio turbulento en el día 5. Así mismo, durante los días 2 y 3 ocurre flujo de calor sensible desde la atmósfera hacia la superficie (la atmósfera está más caliente que la superficie).